## 3.1 Tiling Calculations

In [3]:
# imports
import numpy as np
import matplotlib.pyplot as plt

# scaling constants
B = 8
KB = 1024
MB = KB * 1024
GB = MB * 1024

# constants
R = 3
S = 3
C = 3
M = 64

P = 254
Q = 254

L = 16
T = 9


In [9]:
"""_summary_
R = filter height
S = filter width
C = input channels
M = filter (output) channels
P = output feature map height
Q = output feature map width
L = cache line size (4B for fp32, 1B for uint8)
T = tile size (4B for fp32, 1B for uint8)
"""
def get_tiling_comparison(R, S, C, M, P, Q, L, T):
    cache_miss_wo_tiling = ((C*R*S)/L) * (P*Q*M) + C*R*S*P*Q*M + ((P*Q*M)/L)

    cache_miss_w_tiling = (2*C*R*S*P*Q*M)/(L*T) + (P*Q*M)/L

    print(f"cache misses without tiling: {round(cache_miss_wo_tiling, 3)}")
    print(f"cache misses with tiling: {round(cache_miss_w_tiling, 3)}")

    factor = round(cache_miss_wo_tiling / cache_miss_w_tiling, 3)
    print(f"factor: {factor}")
    
    # * multiply T by 4 for fp32, 1 for uint8
    mem_use = round((3 * ((T*4)**2)) / KB, 3)
    print(f"mem use: {mem_use}KB")
    
    # horizontal/vertical tile loads weights
    # h_tiles_w = (C*R*S) / T
    # v_tiles_w = M / T
    
    # # horizontal/vertical tile loads input feature maps
    # h_tiles_if = (P*Q) / T
    # v_tiles_if = (C*R*S) / T
    
    # # horizontal/vertical tile loads output feature maps
    # h_tiles_of = (P*Q) / T
    # v_tiles_of = M / T

    # tile_loads = round(h_tiles_w * v_tiles_w + h_tiles_if * v_tiles_if + h_tiles_of * v_tiles_of, 3)
    # print(f"min tile loads: {tile_loads}")

# Conv 1
print("Conv 1 L1D")
get_tiling_comparison(5, 5, 3, 32, 60, 60, 16, 25)
print()

print("Conv 1 L1D, L2")
get_tiling_comparison(5, 5, 3, 32, 60, 60, 16, 150)
print()

print("Conv 1 L1D, L2, L3")
get_tiling_comparison(5, 5, 3, 32, 60, 60, 16, 600)
print()


# Conv 2
print("Conv 2 L1D")
get_tiling_comparison(5, 5, 32, 32, 56, 56, 16, 25)
print()

print("Conv 2 L1D, L2")
get_tiling_comparison(5, 5, 32, 32, 56, 56, 16, 100)
print()

print("Conv 2 L1D, L2, L3")
get_tiling_comparison(5, 5, 32, 32, 56, 56, 16, 800)
print()


Conv 1 L1D
cache misses without tiling: 9187200.0
cache misses with tiling: 50400.0
factor: 182.286
mem use: 29.297KB

Conv 1 L1D, L2
cache misses without tiling: 9187200.0
cache misses with tiling: 14400.0
factor: 638.0
mem use: 1054.688KB

Conv 1 L1D, L2, L3
cache misses without tiling: 9187200.0
cache misses with tiling: 9000.0
factor: 1020.8
mem use: 16875.0KB

Conv 2 L1D
cache misses without tiling: 85305472.0
cache misses with tiling: 407680.0
factor: 209.246
mem use: 29.297KB

Conv 2 L1D, L2
cache misses without tiling: 85305472.0
cache misses with tiling: 106624.0
factor: 800.059
mem use: 468.75KB

Conv 2 L1D, L2, L3
cache misses without tiling: 85305472.0
cache misses with tiling: 18816.0
factor: 4533.667
mem use: 30000.0KB

